In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv("galaxy_users.csv")
df.head(2)

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService   
0  7590-VHVEG  Female              0     Yes         No       1           No  \
1  5575-GNVDE    Male              0      No         No      34          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection   
0  No phone service             DSL             No  ...               No  \
1                No             DSL            Yes  ...              Yes   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling   
0          No          No              No  Month-to-month              Yes  \
1          No          No              No        One year               No   

      PaymentMethod MonthlyCharges  TotalCharges  Churn  
0  Electronic check          29.85         29.85     No  
1      Mailed check          56.95       1889.50     No  

[2 rows x 21 columns]

### Q1.

In [3]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [4]:
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].copy()
df_q1.head(2)

OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV   
0             No          Yes               No          No          No  \
1            Yes           No              Yes          No          No   

  StreamingMovies  
0              No  
1              No

In [5]:
df_q1 = df_q1.replace({"Yes": 1, "No": 0})
df_q1.head(2)

OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV   
0              0            1                0           0           0  \
1              1            0                1           0           0   

  StreamingMovies  
0               0  
1               0

In [6]:
df_q1["OnlineSecurity"].unique()

array([0, 1, 'No internet service'], dtype=object)

In [7]:
for n in range(5):
    print(df_q1.iloc[:, n].unique())

[0 1 'No internet service']
[1 0 'No internet service']
[0 1 'No internet service']
[0 1 'No internet service']
[0 1 'No internet service']


In [8]:
# df_q1.unique()
# df_q1.drop_duplicates()
# df_q1.apply(lambda x: x.unique())
df_q1.apply(lambda x: [x.unique()]) # pandas 시험버전

OnlineSecurity                 OnlineBackup   
0  [0, 1, No internet service]  [1, 0, No internet service]  \

              DeviceProtection                  TechSupport   
0  [0, 1, No internet service]  [0, 1, No internet service]  \

                   StreamingTV              StreamingMovies  
0  [0, 1, No internet service]  [0, 1, No internet service]

In [9]:
df_q1.apply(lambda x: [x.unique()]).iloc[0, ].explode().reset_index()

index                    0
0     OnlineSecurity                    0
1     OnlineSecurity                    1
2     OnlineSecurity  No internet service
3       OnlineBackup                    1
4       OnlineBackup                    0
5       OnlineBackup  No internet service
6   DeviceProtection                    0
7   DeviceProtection                    1
8   DeviceProtection  No internet service
9        TechSupport                    0
10       TechSupport                    1
11       TechSupport  No internet service
12       StreamingTV                    0
13       StreamingTV                    1
14       StreamingTV  No internet service
15   StreamingMovies                    0
16   StreamingMovies                    1
17   StreamingMovies  No internet service

In [10]:
df_q1_sub = df_q1.loc[df_q1["OnlineSecurity"] != "No internet service", ]
df_q1_sub.head(1)

OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV   
0              0            1                0           0           0  \

  StreamingMovies  
0               0

In [11]:
df_q1_sub.apply(lambda x: x.unique())

OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV   
0              0            1                0           0           0  \
1              1            0                1           1           1   

  StreamingMovies  
0               0  
1               1

In [12]:
df_q1.loc[df_q1["OnlineSecurity"] == "No internet service", ]

OnlineSecurity         OnlineBackup     DeviceProtection   
11    No internet service  No internet service  No internet service  \
16    No internet service  No internet service  No internet service   
21    No internet service  No internet service  No internet service   
22    No internet service  No internet service  No internet service   
33    No internet service  No internet service  No internet service   
...                   ...                  ...                  ...   
7006  No internet service  No internet service  No internet service   
7008  No internet service  No internet service  No internet service   
7009  No internet service  No internet service  No internet service   
7019  No internet service  No internet service  No internet service   
7026  No internet service  No internet service  No internet service   

              TechSupport          StreamingTV      StreamingMovies  
11    No internet service  No internet service  No internet service  
16    No internet service  No internet service  No internet service  
21    No internet service  No internet service  No internet service  
22    No internet service  No internet service  No internet service  
33    No internet service  No internet service  No internet service  
...                   ...                  ...                  ...  
7006  No internet service  No internet service  No internet service  
7008  No internet service  No internet service  No internet service  
7009  No internet service  No internet service  No internet service  
7019  No internet service  No internet service  No internet service  
7026  No internet service  No internet service  No internet service  

[1520 rows x 6 columns]

In [14]:
# df_q1_sub.sum(axis = 1)
df_q1_sub.sum(axis = 1).value_counts()

3    1117
2    1033
1     966
4     850
0     693
5     569
6     284
Name: count, dtype: int64

In [20]:
966 / 284

In [22]:
round(966 / 284, 1)

In [18]:
val_1 = df_q1_sub.sum(axis = 1).value_counts()[1]
val_6 = df_q1_sub.sum(axis = 1).value_counts()[6]

In [19]:
round(val_1 / val_6, 1)

3.4

In [24]:
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].copy()

In [26]:
df_q1.apply(lambda x: sum(x == "Yes"), axis = 1).value_counts()

0    2213
3    1117
2    1033
1     966
4     850
5     569
6     284
Name: count, dtype: int64

### Q2.

In [29]:
12 // 5, 19 // 5

(2, 3)

In [30]:
df_q2 = df[["tenure", "MonthlyCharges", "TotalCharges"]].copy()
df_q2.head(1)

tenure  MonthlyCharges  TotalCharges
0       1           29.85         29.85

In [31]:
df_q2["month"] = df_q2["TotalCharges"] // df_q2["MonthlyCharges"]
df_q2.head(2)

tenure  MonthlyCharges  TotalCharges  month
0       1           29.85         29.85    1.0
1      34           56.95       1889.50   33.0

In [33]:
df_q2.iloc[:, [0, 1, 3]].corr().abs().round(3)

tenure  MonthlyCharges  month
tenure           1.000           0.247  0.999
MonthlyCharges   0.247           1.000  0.246
month            0.999           0.246  1.000

### Q3.

In [35]:
col1 = ["SeniorCitizen", "Partner", "Dependents", "tenure", "MonthlyCharges", "TotalCharges"]
col2 = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingMovies", "PaperlessBilling"]
df_q3 = df[["Churn"] + col1 + col2].copy()
df_q3.head(2)

Churn  SeniorCitizen Partner Dependents  tenure  MonthlyCharges   
0    No              0     Yes         No       1           29.85  \
1    No              0      No         No      34           56.95   

   TotalCharges OnlineSecurity OnlineBackup DeviceProtection TechSupport   
0         29.85             No          Yes               No          No  \
1       1889.50            Yes           No              Yes          No   

  StreamingMovies PaperlessBilling  
0              No              Yes  
1              No               No

In [34]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [37]:
df_q3 = df_q3.replace({"Yes": 1, "No": 0})

In [40]:
df_q3.loc[:, "OnlineSecurity":"StreamingMovies"].apply(lambda x: x.unique())

In [41]:
df_q3 = df_q3.replace({"No internet service": -1})

In [ ]:
# ser = pd.Series(["a", "b"])
# ser.replace(to_replace = "a", value = "kkk")

In [42]:
df_q3.dtypes

Churn                 int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
MonthlyCharges      float64
TotalCharges        float64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingMovies       int64
PaperlessBilling      int64
dtype: object

In [46]:
df_train, df_test = train_test_split(df_q3, train_size = 0.7, random_state = 123)
len(df_train), len(df_test)

(4922, 2110)

In [48]:
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor  = model_nor.transform(df_test)

In [49]:
arr_train_nor[:2, ]

array([[1.        , 0.        , 0.        , 0.        , 0.08450704,
        0.81116094, 0.07551927, 0.5       , 1.        , 0.5       ,
        0.5       , 1.        , 1.        ],
       [1.        , 0.        , 1.        , 0.        , 0.        ,
        0.60737419, 0.00698708, 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 1.        ]])

In [58]:
model_nor.data_max_, model_nor.data_min_

(array([1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 7.2000e+01,
        1.1875e+02, 8.6848e+03, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00]),
 array([ 0. ,  0. ,  0. ,  0. ,  1. , 18.4, 18.8, -1. , -1. , -1. , -1. ,
        -1. ,  0. ]))

In [56]:
# # 시험버전
# col_types = df.dtypes.astype("str")
# col_types[col_types.str.contains("[0-9]$")].index

# # 1.0.0 이상 버전
# df.head(2).select_dtypes(exclude = "number")

In [59]:
df_train_nor = pd.DataFrame(arr_train_nor, 
                            columns = df_train.columns)
df_train_nor.head(2)

Churn  SeniorCitizen  Partner  Dependents    tenure  MonthlyCharges   
0    1.0            0.0      0.0         0.0  0.084507        0.811161  \
1    1.0            0.0      1.0         0.0  0.000000        0.607374   

   TotalCharges  OnlineSecurity  OnlineBackup  DeviceProtection  TechSupport   
0      0.075519             0.5           1.0               0.5          0.5  \
1      0.006987             0.5           0.5               0.5          0.5   

   StreamingMovies  PaperlessBilling  
0              1.0               1.0  
1              0.5               1.0

In [61]:
model_lr = LogisticRegression(random_state = 123)
model_lr.fit(X = arr_train_nor[:, 1:],
             y = arr_train_nor[:, 0])
pred = model_lr.predict(arr_test_nor[:, 1:])
pred[:3]

array([0., 0., 0.])

In [62]:
round(f1_score(y_true = arr_test_nor[:, 0], 
               y_pred = pred), 2)

0.55

### Q. 특정 변수를 제외한 많은 범주를 한 번에 원하는 값으로 치환을 하는 경우?

In [63]:
df = pd.read_csv("../diamonds.csv")
df.head(2)

carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31

In [65]:
df_sub = df[["cut", "color", "clarity"]].copy()
df_sub.head(2)

cut color clarity
0    Ideal     E     SI2
1  Premium     E     SI1

In [66]:
df_sub.apply(lambda x: x.unique())

cut         [Ideal, Premium, Good, Very Good, Fair]
color                         [E, I, J, H, F, G, D]
clarity    [SI2, SI1, VS1, VS2, VVS2, VVS1, I1, IF]
dtype: object

In [70]:
ser_cat = df_sub.apply(lambda x: x.unique()).explode()
np.where(ser_cat.isin(["E", "H"]), ser_cat, -1)

array([-1, -1, -1, -1, -1, 'E', -1, -1, 'H', -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1], dtype=object)

In [71]:
ser_repl = pd.Series(np.where(ser_cat.isin(["E", "H"]), ser_cat, -1),
                     index = ser_cat)
ser_repl

Ideal        -1
Premium      -1
Good         -1
Very Good    -1
Fair         -1
E             E
I            -1
J            -1
H             H
F            -1
G            -1
D            -1
SI2          -1
SI1          -1
VS1          -1
VS2          -1
VVS2         -1
VVS1         -1
I1           -1
IF           -1
dtype: object

In [72]:
df_sub2 = df_sub.replace(ser_repl)

In [73]:
df_sub2.apply(lambda x: x.unique())

cut              [-1]
color      [E, -1, H]
clarity          [-1]
dtype: object

### Q.

In [75]:
df = pd.read_csv("galaxy_users.csv")
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].copy()
df_q1.head(2)

OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV   
0             No          Yes               No          No          No  \
1            Yes           No              Yes          No          No   

  StreamingMovies  
0              No  
1              No

In [76]:
df_q1 = df_q1.replace({"Yes": np.nan})
df_q1 = df_q1.dropna()
df_q1.head()

OnlineSecurity         OnlineBackup     DeviceProtection   
4                    No                   No                   No  \
11  No internet service  No internet service  No internet service   
16  No internet service  No internet service  No internet service   
21  No internet service  No internet service  No internet service   
22  No internet service  No internet service  No internet service   

            TechSupport          StreamingTV      StreamingMovies  
4                    No                   No                   No  
11  No internet service  No internet service  No internet service  
16  No internet service  No internet service  No internet service  
21  No internet service  No internet service  No internet service  
22  No internet service  No internet service  No internet service